In [1]:
from algorithm.utils.data.tf.dataimage import DataImage
import tensorflow as tf
import keras
%load_ext tensorboard

In [2]:
dataset = DataImage(data_path="Input/dataset", split=0.2, resize=True, height='auto', width='auto', normalize=True, mean='auto', name="Electronic components dataset", batch_size=16)

In [3]:
dataset

Electronic components dataset
	Number of points: 1223
	Root location: Input/dataset
	Split: 0.2
	Transform used:	Resize(size=(128, 128)
	Normalize(mean=[0.00392156862745098, 0.00392156862745098, 0.00392156862745098], std=None)


In [4]:
trainset = dataset.get_set(split="train")
validset = dataset.get_set(split="validation")

In [5]:
# get some random training images
dataiter = iter(trainset)
images, labels = next(dataiter)

In [6]:
images.shape, len(labels)

(TensorShape([16, 128, 128, 3]), 16)

In [7]:
images

<tf.Tensor: shape=(16, 128, 128, 3), dtype=float32, numpy=
array([[[[0.05882353, 0.13725491, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608],
         ...,
         [0.06170344, 0.15006128, 0.09803922],
         [0.05741422, 0.13866423, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608]],

        [[0.05882353, 0.13725491, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608],
         ...,
         [0.06170344, 0.15006128, 0.09803922],
         [0.05741422, 0.13866423, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608]],

        [[0.05882353, 0.13725491, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608],
         ...,
         [0.06170344, 0.15006128, 0.09803922],
         [0.05741422, 0.13866423, 0.09019608],
         [0.05882353, 0.13725491, 0.09019608]],

        ...,

        [[0.05882353, 0.13725491,

In [8]:
labels

<tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'PINTH', b'PINTH', b'PINTH', b'PINTH', b'PINTH', b'0805S',
       b'STRIP6', b'CAPACITOR', b'CAPACITOR', b'X4Y18', b'PINTH',
       b'CAPACITOR', b'R20', b'JUMPER', b'CAPACITOR', b'PINTH'],
      dtype=object)>

In [9]:
class_names = dataset.labels

In [10]:
class_names

['X18Y4',
 'CONNECTOR',
 'JUMPER',
 'X43Y55',
 'C1210',
 'S5SOT23',
 'TRIAC',
 'STRIP3',
 'R01005',
 'OSCILLATOR',
 'WSK2512',
 'STRIP6',
 'R63H40',
 'PINSMD',
 'R20',
 'RELAIS',
 'SMB',
 'PINTH',
 'X8Y11',
 'FUSE',
 'C0805',
 'DIODE',
 'LED3',
 'LED',
 'X63H40',
 'DIGITAL',
 'RESISTOR',
 'R1206',
 'CASEC',
 'C0603',
 'OPTOISOLATOR',
 'PLCC2LED',
 'TRANSFORMER',
 'X50Y65',
 'OPENFIX',
 'SO8',
 'MOS',
 'SFUSE',
 'INDUCTANCE',
 'AUTIC',
 'R0603',
 '0805S',
 'SO16',
 'GATE',
 'SCR',
 'R0805',
 'ZENER',
 'CAPACITOR',
 'X4Y18',
 'R2512',
 'VOLTAGEREGULATOR',
 'MINIMELF',
 'R110H126',
 'TRANSISTOR',
 'R0402',
 'CTELRJ45',
 'R10']

In [ ]:
import matplotlib.pyplot as plt
image_batch, label_batch = next(iter(trainset))

plt.figure(figsize=(60, 60))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8")) # to be implemented: normalization and resize
    label = label_batch[i].numpy().decode("utf-8")
    plt.title(label, fontsize=60)
    plt.axis("off")

In [12]:
from algorithm.deep.tf.neural_networks import ResNet2__0

In [13]:
trainset = dataset.apply_one_hot_encoding(trainset)
validset = dataset.apply_one_hot_encoding(validset)

In [14]:
out_dir = "out\models\ResNet2__0"

In [15]:
model = ResNet2__0(len(class_names),input_shape=(128, 128, 3))
tb_callback = keras.callbacks.TensorBoard(log_dir=f'{out_dir}/tensorboard_log', histogram_freq=1)

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
    run_eagerly=True,
)

In [17]:
model.fit(trainset, validation_data=validset, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
62/62 [==============================] - 240s 4s/step - loss: 6.2506 - accuracy: 0.4239 - val_loss: 15301369.0000 - val_accuracy: 0.0164
Epoch 2/1000
62/62 [==============================] - 239s 4s/step - loss: 1.9018 - accuracy: 0.5250 - val_loss: 2.9513 - val_accuracy: 0.1311
Epoch 3/1000
62/62 [==============================] - 245s 4s/step - loss: 1.3893 - accuracy: 0.6180 - val_loss: 3.0543 - val_accuracy: 0.1311
Epoch 4/1000
62/62 [==============================] - 245s 4s/step - loss: 1.3252 - accuracy: 0.6374 - val_loss: 3.4649 - val_accuracy: 0.1311
Epoch 5/1000
62/62 [==============================] - 245s 4s/step - loss: 1.1164 - accuracy: 0.6864 - val_loss: 3.5832 - val_accuracy: 0.1311
Epoch 6/1000
62/62 [==============================] - 245s 4s/step - loss: 0.9756 - accuracy: 0.7058 - val_loss: 3.7909 - val_accuracy: 0.2910
Epoch 7/1000
62/62 [==============================] - 244s 4s/step - loss: 0.9011 - accuracy: 0.7324 - val_loss: 3.9741 - val_accuracy:

In [18]:
model.summary()

Model: "res_net2__0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 4, 4, 2048)        58370944  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 1000)              32769000  
                                                                 
 dense_1 (Dense)             (None, 57)                57057     
                                                                 
 model (Functional)          (None, 57)                91197001  
                                                                 
Total params: 91197001 (347.89 MB)
Trainable params: 91045577 (347.31 MB)
Non-trainable params: 151424 (591.50 KB)
_________________________________________________________________


In [19]:
model.save(out_dir)

INFO:tensorflow:Assets written to: out\models\ResNet2__0\assets


INFO:tensorflow:Assets written to: out\models\ResNet2__0\assets


In [ ]:
keras.utils.plot_model(model, f'{out_dir}/ResNet2__0.svg', show_shapes=True, show_layer_activations=True, show_layer_names=True)

In [21]:
# Evaluation on unseen data.

test_loss, test_acc = model.evaluate(validset, verbose=2)
print('\nTest accuracy:', test_acc)

16/16 - 11s - loss: 6.2021 - accuracy: 0.6885 - 11s/epoch - 711ms/step

Test accuracy: 0.688524603843689
